# Make Visual Words

## Rquirements

k means clustering is memory intensive process. 

## Input

* n-dim descriptors
    * For making visual words, which image contains which descriptor is not important. We just do quantization.

* Number of visual words

## Process

Build cluster.
Say, we use k-means clustering. We determine number of clusters. 
Run k-means clustering. 
We can accelerate speed by using approximate nearest neighbor search such as kd-tree, but we lose accuracy. 

## Output

index of visual words, and its cluster's center point. 

From a given feature, we can use the output information to determine most closest visual words. 



In [ ]:
# Load all features in the dataset. 
# Q. What if there is excatly same feature many times? do we add it into feature pool? 
# Q. Does many same feature point affect the result of k-menas clustering?

# An image may contain several features, and we need to get all features. 

%load_ext autoreload
%autoreload 2

from src import visual_word_generation

config_path='./config/visual_word_gen.config'
config = visual_word_generation.read_config(config_path)
visual_word_generation.main(config)

In [ ]:
%%time
import pyflann

flann = pyflann.FLANN(cores=5)

# max_iter = config['flann_iter']
max_iter = 1
print("run k-means clustering with FLANN. max_itre:{}".format(max_iter))
centers = flann.kmeans(data_points, k, max_iter, cores=0)

----------- Below codes are legacy ------------------

In [ ]:
%%time
clustering_utils.run_kmeans_clustering(data_points, k, init_center_points=center_points, knn_method="naive")

# Way to speed up codebook learning

* [Annoy]() to assign centroids for all features
* FLANN vs ANNOY


In [ ]:
%%time
center_points = clustering_utils.run_kmeans_clustering(data_points, k, max_iter=15, init_center_points=center_points, knn_method="kdtree")

In [ ]:
%%time
clustering_utils.run_kmeans_clustering(data_points.astype(float), k, init_center_points=center_points, knn_method="kdtree")

In [ ]:
%%time
clustering_utils.run_kmeans_clustering(data_points, k, init_center_points=center_points, knn_method="kdtree")